In [1]:
import pandas as pd
import folium
import folium.plugins as plugins
from tqdm import tqdm

In [2]:
df = pd.read_csv('shops.tsv',sep='\t')

In [3]:
list(set(df[df['name'].apply(lambda x: 'dns' in x.lower())]['name']))

['DNS TechnoPoint',
 'Магазин электроники DNS',
 'DNS',
 'DNS цифровая и бытовая техника',
 'Цифровой супермаркет DNS',
 'Cifrovoj supermarket DNS']

In [4]:
we = ['М.Видео','Эльдорадо','Media Markt']
main_competitors = ['DNS цифровая и бытовая техника',
 'Цифровой супермаркет DNS',
 'DNS TechnoPoint',
 'Cifrovoj supermarket DNS',
 'DNS',
 'Магазин электроники DNS']

In [5]:
df['class'].value_counts()

Салон связи            3859
Магазин электроники    3598
Name: class, dtype: int64

In [6]:
lat_max, lon_max = 55.974806, 38.011751

lat_min, lon_min = 55.615999, 37.131547


In [7]:
filt1 = df['lat']>=lat_min
filt2 = df['lat']<=lat_max
filt3 = df['lon']>=lon_min
filt4 = df['lon']<=lon_max
filt = filt1&filt2&filt3&filt4
df = df[filt]

In [8]:
df_homes = pd.read_excel('Жилые дома с числом квартир и ценой v4.xlsx')

filt1 = df_homes['lat']>=lat_min
filt2 = df_homes['lat']<=lat_max
filt3 = df_homes['lon']>=lon_min
filt4 = df_homes['lon']<=lon_max
filt = filt1&filt2&filt3&filt4
df_homes = df_homes[filt]


min_lat_kr, min_lon_kr = 55.752600, 37.614400
max_lat_kr, max_lon_kr = 55.756871, 37.627532
filt1 = df_homes['lat']>=min_lat_kr
filt2 = df_homes['lat']<=max_lat_kr
filt3 = df_homes['lon']>=min_lon_kr
filt4 = df_homes['lon']<=max_lon_kr
filt = filt1&filt2&filt3&filt4
filt = filt==False
df_homes = df_homes[filt]

In [9]:
map_ = folium.Map(location = list(df[['lat','lon']].mean().values), zoom_start=12)
print(len(df))
for i, row in tqdm(df.iterrows()):
        radius = 5
        if  row['name'] in we:
            color = 'green'
            radius = 10
        elif row['class']=='Магазин электроники':
            color = 'red'
        else:
            color ='brown'
        if  row['name'] in main_competitors:
            radius = 10
        folium.CircleMarker(row[['lat','lon']].values,fill=True,
                            fill_opacity = 1,
                            color=color,
                            opacity=0,
                            radius = radius,
                            popup =  row['name']
                           ).add_to(map_)

8it [00:00, 70.57it/s]

3194


3194it [00:51, 62.58it/s]


In [10]:
data = df_homes[['lat','lon','flat_num']].as_matrix()
plugins.HeatMap(list(map(list,data)),radius=10,blur=20).add_to(map_)


/home/tserekh/jupnb/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [11]:
map_.save('map.html')